In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.chdir("..")

# import sys
# sys.path.append("./vrevals")

import re
import ast
import json
import numpy as np
import pandas as pd
import yaml
import time
import glob

import matplotlib.pyplot as plt

from collections import Counter
from utils.math_equivalence import is_math_equiv, extract_boxed_answer
from utils.pass_k_utils import estimate_pass_at_k

In [11]:
class Args:
    dataset_name = "imobench"
    split = "answerbench_subset"
    k_list = [1,4,8,32]
    subset_num = None
    step_by_step_prompt = True
    n_threads = 1
args = Args()

job_dir = f"runs/v1/{args.dataset_name}.qwen2.5-math-7b"
sampler_config_dir = f'{job_dir}/rlzero.direct/sample_1'
# sampler_config_dir = f'{job_dir}/instruct.prefix/Anya'
# sampler_config_dir = f'{job_dir}/instruct.prefix/Anya'
# sampler_config_dir = f'{job_dir}/instruct.prefix/Becky'

prompt_csv_path = f'{job_dir}/{args.split}.prompts.csv'

# with open(f"{sampler_config_dir}/sampler_config.yaml", "r") as f:
#     sampler_config = yaml.safe_load(f)
# sampler_config

print(sampler_config_dir)

runs/v1/imobench.qwen2.5-math-7b/rlzero.direct/sample_1


In [12]:
# all_generation_csv = glob.glob(f"{sampler_config_dir}/{args.split}.generations.*.csv")
all_generation_csv = glob.glob(f"{sampler_config_dir}/*generations*.csv")
all_generation_df = [pd.read_csv(p) for p in all_generation_csv]
# Concatenate all dataframes in all_generation_df into a single dataframe
generation_df = pd.concat(all_generation_df, ignore_index=True)
generation_df['pred_answer'] = generation_df.response.apply(extract_boxed_answer)
generation_df['is_valid'] = generation_df.pred_answer.apply(lambda x: len(x) > 0)
generation_df['is_correct'] = generation_df.apply(lambda row: is_math_equiv(row['gt_answer'], row['pred_answer']), axis=1)

In [13]:
generation_df

,question_id,prompt_id,response,pred_answer,gt_answer,sampler_config,is_valid,is_correct
0,imo-bench-geometry-010,imo-bench-geometry-010,"To solve this problem, we need to find the cir...",\frac{p}{2},$\frac{pq}{p+q}$,"{'sampler': {'class': 'VLLMSampler', 'model_na...",True,False
1,imo-bench-geometry-010,imo-bench-geometry-010,"To solve this problem, we need to find the cir...",\frac{q^2}{2\sqrt{4q^2 - p^2}},$\frac{pq}{p+q}$,"{'sampler': {'class': 'VLLMSampler', 'model_na...",True,False
2,imo-bench-geometry-010,imo-bench-geometry-010,"To solve this problem, we need to find the cir...",\frac{q^2}{2\sqrt{4q^2 - p^2}},$\frac{pq}{p+q}$,"{'sampler': {'class': 'VLLMSampler', 'model_na...",True,False
3,imo-bench-geometry-010,imo-bench-geometry-010,"To solve this problem, we need to find the cir...",\frac{p^2 (p + 2q)^2 (4q^2 - p^2) (p + q)}{32},$\frac{pq}{p+q}$,"{'sampler': {'class': 'VLLMSampler', 'model_na...",True,False
4,imo-bench-geometry-010,imo-bench-geometry-010,"To solve this problem, we need to find the cir...",\frac{p}{2},$\frac{pq}{p+q}$,"{'sampler': {'class': 'VLLMSampler', 'model_na...",True,False
...,...,...,...,...,...,...,...,...
40995,imo-bench-combinatorics-042,imo-bench-combinatorics-042,"To solve this problem, we need to maximize the...",48762507,4769280,"{'sampler': {'class': 'VLLMSampler', 'model_na...",True,False
40996,imo-bench-combinatorics-042,imo-bench-combinatorics-042,"To solve this problem, we need to maximize the...",49031400,4769280,"{'sampler': {'class': 'VLLMSampler', 'model_na...",True,False
40997,imo-bench-combinatorics-042,imo-bench-combinatorics-042,"To solve this problem, we need to maximize the...",49109460,4769280,"{'sampler': {'class': 'VLLMSampler', 'model_na...",True,False
40998,imo-bench-combinatorics-042,imo-bench-combinatorics-042,"To solve this problem, we need to maximize the...",49092060,4769280,"{'sampler': {'class': 'VLLMSampler', 'model_na...",True,False


In [14]:
# grouped_df = generation_df.groupby(['question_id', 'prompt_id'])

# valid_grouped = grouped_df.is_valid.apply(list).reset_index(name='valids')
# valid_grouped['num_samples'] = valid_grouped.valids.apply(len)
# valid_grouped['num_math_equal'] = valid_grouped.valids.apply(sum)

# min_num_samples = valid_grouped['num_samples'].min()
# k_list = range(1, 32)
# detail_pass_at_k = {
#     f"pass@{k}": estimate_pass_at_k(valid_grouped['num_samples'].values, 
#                                     valid_grouped['num_math_equal'].values, k)
#     for k in k_list
#     if (min_num_samples >= k).all()
# }
# pass_at_k = {k: detail_pass_at_k[k].mean() for k in detail_pass_at_k}
# pass_at_k

In [15]:
# 'pass@1': np.float64(0.09619140624999997),
#  'pass@2': np.float64(0.13234537919517758),
#  'pass@3': np.float64(0.1530545528388288),
#  'pass@4': np.float64(0.1680069818866373),
#  'pass@5': np.float64(0.1800541327713428),
#  'pass@6': np.float64(0.19034425633203222),
#  'pass@7': np.float64(0.19944989789505205),
#  'pass@8': np.float64(0.20769298662495453),
#  'pass@9': np.float64(0.21526857823802353),
#  'pass@10': np.float64(0.22230197162977028),
#  'pass@11': np.float64(0.2288780607952665),
#  'pass@12': np.float64(0.2350573633275176),
#  'pass@13': np.float64(0.24088513783661503),
#  'pass@14': np.float64(0.24639674153141536),
#  'pass@15': np.float64(0.2516208693711693),
#  'pass@16': np.float64(0.25658156415546324),
#  'pass@17': np.float64(0.26129949503265304),
#  'pass@18': np.float64(0.265792790609523),

In [16]:
# Filter the dataframe to keep only rows where the response is valid
# valid_generation_df = generation_df[generation_df['is_valid']]
valid_generation_df = generation_df

cut_off_size = 1024

grouped_df = valid_generation_df.groupby(['question_id', 'prompt_id'])
corrects_grouped = grouped_df.is_correct.apply(list).reset_index(name='corrects')
corrects_grouped.corrects = corrects_grouped.corrects.apply(lambda x: x[:cut_off_size])
corrects_grouped['num_samples'] = corrects_grouped.corrects.apply(len)
corrects_grouped['num_math_equal'] = corrects_grouped.corrects.apply(sum)

min_num_samples = corrects_grouped['num_samples'].min()

k_list = range(1,2000)
detail_pass_at_k = {
    f"pass@{k}": estimate_pass_at_k(corrects_grouped['num_samples'].values, 
                                    corrects_grouped['num_math_equal'].values, k)
    for k in k_list
    if (min_num_samples >= k).all()
}
pass_at_k = {k: detail_pass_at_k[k].mean() for k in detail_pass_at_k}
pass_at_k

{'pass@1': np.float64(0.11396484374999993),
 'pass@2': np.float64(0.16655545500366564),
 'pass@3': np.float64(0.19832546888853939),
 'pass@4': np.float64(0.22043767767971856),
 'pass@5': np.float64(0.23687115620790894),
 'pass@6': np.float64(0.24953349814546927),
 'pass@7': np.float64(0.25955186955259624),
 'pass@8': np.float64(0.26766549418980234),
 'pass@9': np.float64(0.2743822197765478),
 'pass@10': np.float64(0.28005905518433694),
 'pass@11': np.float64(0.28495061158284085),
 'pass@12': np.float64(0.2892405199075973),
 'pass@13': np.float64(0.29306254193011866),
 'pass@14': np.float64(0.2965150332591834),
 'pass@15': np.float64(0.299670983195627),
 'pass@16': np.float64(0.3025850582346557),
 'pass@17': np.float64(0.30529858924914965),
 'pass@18': np.float64(0.30784313214475856),
 'pass@19': np.float64(0.3102430293294173),
 'pass@20': np.float64(0.3125172651690625),
 'pass@21': np.float64(0.31468081857309604),
 'pass@22': np.float64(0.31674565477479666),
 'pass@23': np.float64(0.31

In [17]:
len(pass_at_k)

1024

In [18]:
# {'pass@1': np.float64(0.694953077837839),
#  'pass@2': np.float64(0.8097839655079988)}

In [19]:
overall_results = {
    'detail_pass_at_k': {k:v.tolist() for k,v in detail_pass_at_k.items()},
    'pass_at_k': pass_at_k,
}
final_metrics = {'overall': overall_results}
t = time.localtime()
metrics_json_name = f'metrics.{t.tm_mon}.{t.tm_mday},{t.tm_hour}:{t.tm_min}.json'
with open(os.path.join(sampler_config_dir, metrics_json_name), mode='w', encoding='utf-8') as json_file:
    json.dump(final_metrics, json_file, indent=4, ensure_ascii=False)


In [22]:
final_metrics

{'overall': {'detail_pass_at_k': {'pass@1': [0.37207031249999967,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.2509765624999999,
    0.054687499999999334,
    0.0,
    0.001953125,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.014648437499999889,
    0.8066406249999996,
    0.0,
    0.0,
    0.7451171874999994,
    0.0,
    0.0,
    0.2373046875,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.27148437500000044,
    0.0,
    0.05761718750000011,
    0.0,
    0.0,
    0.00390625,
    0.3017578125000001,
    0.0,
    0.205078125,
    0.0,
    0.0],
   'pass@2': [0.6059326887829911,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.43914765090420327,
    0.10643481182795678,
    0.0,
    0.0039043407869012725,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.02909640762463317,
    0.9627646169354839,
    0.0,
    0.0,
    0.9352203995601174,
    0.0,
    0.0,
    0.4184727822580647,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.46945831805962723,
  

In [43]:
responses = grouped_df.response.apply(list).reset_index(name='text')
pred_answers = grouped_df.pred_answer.apply(list).reset_index(name='text')

In [12]:
responses.text[0]

["Okay, so Janet has these ducks that lay 16 eggs each day. Hmm, let me figure out how many eggs she has in total. If she has 16 eggs per day, and she's selling them at the farmers' market for $2 each, then I guess I need to calculate how much she makes from selling the eggs.\n\nFirst, let me think about how many eggs she eats for breakfast. She eats three eggs every morning. So, that's 3 eggs per day. Then, she also bakes muffins for her friends every day, using four eggs. So, that's another 4 eggs per day. \n\nSo, to find the total number of eggs she has left after eating and baking, I should subtract the eggs she eats and the eggs she uses for muffins from the total number of eggs laid each day. \n\nLet me write that down:\n\nTotal eggs laid per day = 16\n\nEggs eaten for breakfast = 3\n\nEggs used for muffins = 4\n\nSo, the number of eggs left would be:\n\nTotal eggs left = Total eggs laid - Eggs eaten - Eggs used\n\nPlugging in the numbers:\n\nTotal eggs left = 16 - 3 - 4\n\nLet m

In [23]:
print(generation_df[~generation_df['is_valid']].response.values[0])

Okay, so Kylar went to the store to buy glasses, and he wants to buy 16 glasses in total. The problem says that one glass costs $5, but every second glass costs only 60% of the price. Hmm, okay, so I need to figure out how much he needs to pay for all 16 glasses.

First, let me break this down. There are 16 glasses in total. The first glass is $5, and each subsequent glass is 60% of the original price. So, the first glass is $5, the second glass is 60% of $5, the third is 60% of $5, and so on, until the sixteenth glass.

Wait, hold on, is that right? The problem says every second glass costs only 60% of the price. So, does that mean the first glass is $5, and then every second glass is 60% of $5? Or is it that every second glass is 60% of the total number of glasses? Hmm, that might be a bit confusing.

Let me read the problem again: "One glass costs $5, but every second glass costs only 60% of the price." So, it seems like the first glass is $5, and each subsequent glass is 60% of the